In [1]:
import os
import sys
import ipympl

In [2]:
print(os.getcwd())
sys.path.insert(0, os.path.join(os.getcwd(), '../'))

/home/haruyaishikawa/dev/RecordWithRS/python/notebook


In [3]:
import numpy as np
from datetime import datetime as dt
from utils import DataManagement

In [4]:
# setup which datetime to get
dm = DataManagement()
after = dt(2018, 7, 23, 14, 0, 0)
before = dt(2018, 7, 23, 14, 1, 0)
datetimes = dm.get_datetimes_in(after, before)

assert len(datetimes) == 1
datetime = datetimes[0]

In [5]:
data_path = dm.get_save_directory(datetime)
data_path = os.path.join(data_path, 'objects')

print(os.listdir(data_path))

['chair', 'cup', 'dining table', 'keyboard', 'laptop', 'person', 'traffic light']


In [55]:
ob_name = 'cup'
ob_path = os.path.join(data_path, ob_name)

In [56]:
files = os.listdir(ob_path)
print(len(files))
filename = files[0]
csv_path = os.path.join(ob_path, filename)

149


In [57]:
np_pc = np.loadtxt(csv_path, delimiter=',')

In [58]:
print(np_pc.shape)
np_pc.mean(0)

(5051, 3)


array([-336.03258092,  244.45331369, 1393.32231241])

In [75]:
pc = np.rollaxis(np_pc, 1)
print(len(pc[0]))
print(len(pc[1]))

5051
5051


In [60]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [61]:
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pc[0], pc[1], pc[2])

plt.show()

FigureCanvasNbAgg()

In [62]:
%matplotlib widget

plt.scatter(pc[0], pc[1])
plt.xlabel("x")
plt.ylabel("y")
plt.show()

FigureCanvasNbAgg()

In [63]:
# Translation matrix
P_matrix_filename = os.path.join('../static_data', 'T.csv')
P = np.loadtxt(P_matrix_filename, delimiter=',')
P

array([[ 9.510000e-01,  2.440000e-01, -2.690000e-01, -6.967890e+02],
       [-3.580000e-01,  7.580000e-01, -5.780000e-01,  2.306030e+02],
       [ 6.200000e-02,  6.350000e-01,  7.930000e-01,  1.706461e+03],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

In [64]:
def convert2world(P, coord):
    '''
    Convert from Camera coordniate to World coordinate (according to P)
    '''
    _coord = np.concatenate([np.asarray(coord), [1.000]])
    _P = np.array(P)
    #FIXME: Remove this when P is fixed
    rotate = np.array([[1,0,0,0], [0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
    n = rotate.dot(_coord)
    return _P.dot(n)[:3]

In [65]:
ordered_files = dm.natural_sort(files)
mean_of_points = []
for fn in files:
    csv_path = os.path.join(ob_path, fn)
    points = np.loadtxt(csv_path, delimiter=',')
    mean = points.mean(0)
    real_world = convert2world(P, mean)
    mean_of_points.append(real_world)

In [77]:
mp = np.asarray(mean_of_points)
mp = np.rollaxis(mp, 1)
mp.shape

(3, 149)

In [90]:
%matplotlib widget

plt.subplot(2, 2, 1)
plt.plot(mp[0], mp[1])
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.suptitle('Movement of object', fontsize=16)

plt.subplot(2, 2, 2)
plt.scatter(mp[0], mp[1])
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

FigureCanvasNbAgg()